In [ ]:
import tensorflow as tf
gpus=tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu,True)
print(gpus)
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import os,glob, random
import tensorflow_models as tfm
from ptge import GazeModel, vgg16_processor
tf.__version__

### load gaze model

In [2]:
tf.keras.backend.clear_session()
gaze_model=GazeModel()
print(gaze_model({'face':tf.ones((1,224,224,3)),
        'flipped_face':tf.ones((1,224,224,3)),
        'lefteye':tf.ones((1,36,60,3)),
        'righteye':tf.ones((1,36,60,3)),
        'rotation_matrix':tf.ones((1,9)),
        'eye_coords':tf.ones((1,6)),
        'id':tf.constant([1.])}))
gaze_model.load_weights('best_model_2.h5')

2024-05-19 12:52:44.132776: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907


tf.Tensor([[-0.59598786  0.85539657 -0.2608078 ]], shape=(1, 3), dtype=float32)


### calibration model

In [3]:
transformer = tfm.nlp.models.TransformerEncoder(
    num_layers=6,
    num_attention_heads=4,
    intermediate_size=2048,
    activation='relu',
    dropout_rate=0.0,
    attention_dropout_rate=0.0,
    use_bias=not False,
    norm_first=True,
    norm_epsilon=1e-06,
    intermediate_dropout=0.0,
)

In [4]:
gaze_model.trainable=False
class CalibrationModel(tf.keras.Model):
    def __init__(self):
        super(CalibrationModel,self).__init__()
        self.g_face=gaze_model.g_face
        self.g_eye=gaze_model.g_eye
        self.transformer_stack=transformer
        self.flat=tf.keras.layers.Flatten()
        self.MLP1=tf.keras.Sequential([
            tf.keras.layers.Dense(1280,activation='relu'),
            tf.keras.layers.BatchNormalization(),
            ],name='MLP1')
        self.MLP2=tf.keras.Sequential([
            tf.keras.layers.Dense(1280,activation='relu'),
            tf.keras.layers.BatchNormalization(),
            ],name='MLP2')
        self.output_layer=tf.keras.layers.Dense(6,name='subject_feature')
    def call(self,input_dict):
        face_features=self.g_face(input_dict['face'])
        flipped_face_features=self.g_face(input_dict['flipped_face'])
        left_features=vgg16_processor(input_dict['lefteye'])
        left_features=self.g_eye(left_features)
        right_features=vgg16_processor(input_dict['righteye'])
        right_features=self.g_eye(right_features)
        face_features=self.flat(face_features)
        flipped_face_features=self.flat(flipped_face_features)
        left_features=self.flat(left_features)
        right_features=self.flat(right_features)
        rot_mat=input_dict['rotation_matrix']
        rot_mat_flipped=input_dict['flipped_rotation_matrix']
        eye_coords=input_dict['eye_coords']
        gaze=input_dict['gaze']
        gaze_flipped=input_dict['gaze_flipped']
        total=tf.concat([face_features,flipped_face_features,left_features,
                            right_features,eye_coords,rot_mat,rot_mat_flipped,
                            gaze,gaze_flipped],1)
        total=self.MLP1(total)
        total = tf.expand_dims(total, axis=1) 
        total=self.transformer_stack(total)
        total=self.MLP2(tf.squeeze(total,1))
        final_output=self.output_layer(total)
        return final_output
        

### data for newperson to estimate person speciific preference vector using calibration model

In [3]:
face=[]
lefteye=[]
righteye=[]
rotation_matrix=[]
flipped_rotation_matrix=[]
gaze=[]
gaze_flipped=[]
subject_id=[]
eye_coords=[]
subject_map={'processed_data/Image/p00': 0, 'processed_data/Image/p01': 1}
data_path='processed_data/Image'
persons=os.listdir(data_path)
persons.sort()
person=persons[2]
face+=glob.glob(f'{data_path}/{person}/face/*')
lefteye+=glob.glob(f'{data_path}/{person}/lefteye/*')
righteye+=glob.glob(f'{data_path}/{person}/righteye/*')
rotation_matrix+=glob.glob(f'{data_path}/{person}/rotation_matrix/*')
flipped_rotation_matrix+=glob.glob(f'{data_path}/{person}/rotation_matrix_flipped/*')
gaze+=glob.glob(f'{data_path}/{person}/3d_gaze/*')
gaze_flipped+=glob.glob(f'{data_path}/{person}/3d_gaze_flipped/*')
subject_id+=[f'{data_path}/{person}' for _ in range(len(face))]
eye_coords+=glob.glob(f'{data_path}/{person}/eye_coords/*')
subject_map[f'{data_path}/{person}']=2

face.sort()
lefteye.sort()
righteye.sort()
rotation_matrix.sort()
flipped_rotation_matrix.sort()
gaze.sort()
gaze_flipped.sort()
eye_coords.sort()
subject_id.sort()
data=list(zip(face,lefteye,righteye,rotation_matrix,flipped_rotation_matrix,eye_coords,gaze,gaze_flipped,subject_id))   
random.seed(12)
random.shuffle(data)
data=tf.data.experimental.from_list(data)
print(subject_map)

{'processed_data/Image/p00': 0, 'processed_data/Image/p01': 1, 'processed_data/Image/p02': 2}


In [4]:
subject_map=tf.lookup.StaticHashTable( tf.lookup.KeyValueTensorInitializer(list(subject_map.keys()), 
                                                                           list(subject_map.values())),default_value=-1)

In [5]:
@tf.function
def load_img(img):
    img=tf.io.read_file(img)
    img=tf.io.decode_jpeg(img,3)
    return img
@tf.numpy_function(Tout=tf.float32)
def ld(x):
    return np.load(x).astype('float32').ravel()
@tf.function
def map_fn_calibration(face,
            lefteye,
            righteye,
            rotation_matrix,
            flipped_rotation_matrix,
            eye_coords,
            gaze,
            gaze_flipped,
            subject_id,
            ):
    face=load_img(face)
    flipped_face=tf.image.flip_left_right(face)
    lefteye=load_img(lefteye)
    righteye=load_img(righteye)
    rotation_matrix=ld(rotation_matrix)
    flipped_rotation_matrix=ld(flipped_rotation_matrix)
    eye_coords=ld(eye_coords)
    id=subject_map[subject_id]
    gaze=ld(gaze)
    gaze_flipped=ld(gaze_flipped)
    subject_embedding=gaze_model.embedding(id)
    return {
            'face':face,
            'flipped_face':flipped_face,
            'lefteye':lefteye,
            'righteye':righteye,
            'rotation_matrix':rotation_matrix,
            'flipped_rotation_matrix':flipped_rotation_matrix,
            'eye_coords':eye_coords,
            'gaze':gaze,
            'gaze_flipped':gaze_flipped,
            }
@tf.function
def map_fn_gaze(face,lefteye,righteye,rotation_matrix,eye_coords,subject_id,gaze):
    face=load_img(face)
    flipped_face=tf.image.flip_left_right(face)
    lefteye=load_img(lefteye)
    righteye=load_img(righteye)
    rotation_matrix=ld(rotation_matrix)
    eye_coords=ld(eye_coords)
    id=subject_map[subject_id]
    gaze=ld(gaze)
    return {
            'face':face,
            'flipped_face':flipped_face,
            'lefteye':lefteye,
            'righteye':righteye,
            'rotation_matrix':rotation_matrix,
            'eye_coords':eye_coords,
            'id':id},gaze



### calibration model loading

In [8]:
model=CalibrationModel()
model(next(iter(data.map(map_fn_calibration).batch(1))))
model.load_weights('calibr.h5')

### estimation of person specific embedding for new person p02 using calibration model

In [9]:
new_embeddings=model(next(iter(data.map(map_fn_calibration).batch(1))))
new_embeddings

<tf.Tensor: shape=(1, 6), dtype=float32, numpy=
array([[-0.24915332,  0.6478733 , -1.0298878 , -0.3768737 ,  0.36539593,
        -0.5966729 ]], dtype=float32)>

### updating the person specific embedding table of the gazeModel with the embeddings of new person

In [10]:
mat=gaze_model.embedding.weights[0].numpy()
mat=np.append(mat,new_embeddings,axis=0)
gaze_model.embedding=tf.keras.layers.Embedding(3,6,embeddings_initializer=tf.keras.initializers.Constant(mat),
                                            embeddings_regularizer=tf.keras.regularizers.L2(l2=0.01),
                                            mask_zero=True,name='subject_embedding')

In [12]:
gaze_model.embedding(2)

<tf.Tensor: shape=(6,), dtype=float32, numpy=
array([-0.24915332,  0.6478733 , -1.0298878 , -0.3768737 ,  0.36539593,
       -0.5966729 ], dtype=float32)>

In [9]:
class AngularError(tf.keras.metrics.Metric):

    def __init__(self, name='mean_angular_error', **kwargs):
        super().__init__(name=name, **kwargs)
        self.total_error = self.add_weight(name='total_error', initializer='zeros')
        self.num_samples = self.add_weight(name='num_samples', initializer='zeros')
        
    def update_state(self, y_true, y_pred,sample_weight=None):
        y_true = tf.math.l2_normalize(y_true, axis=-1)
        y_pred = tf.math.l2_normalize(y_pred, axis=-1)
    
        dot_product = tf.reduce_sum(y_true * y_pred, axis=-1)
        dot_product = tf.clip_by_value(dot_product, -1.0, 1.0)
     
        angular_error = tf.acos(dot_product)
        angular_error=angular_error*57.296
        self.total_error.assign_add(tf.reduce_sum(angular_error))
        self.num_samples.assign_add(tf.cast(tf.shape(y_true)[0], tf.float32))

    def result(self):
        return self.total_error / self.num_samples
    def reset_state(self):
        self.total_error.assign(0.0)
        self.num_samples.assign(0.0)


### data of new person to be tested using the gazemodel if the calibration is done properly

In [ ]:
face,lefteye,righteye,rotation_matrix,gaze,subject_id,eye_coords=[],[],[],[],[],[],[]
data_path='processed_data/Image'
persons=os.listdir(data_path)
persons.sort()
print(persons)
person=persons[0]
face+=glob.glob(f'{data_path}/{person}/face/*')
lefteye+=glob.glob(f'{data_path}/{person}/lefteye/*')
righteye+=glob.glob(f'{data_path}/{person}/righteye/*')
rotation_matrix+=glob.glob(f'{data_path}/{person}/rotation_matrix/*')
gaze+=glob.glob(f'{data_path}/{person}/3d_gaze/*')
subject_id+=[f'{data_path}/{person}' for _ in range(len(face))]
eye_coords+=glob.glob(f'{data_path}/{person}/eye_coords/*')
face.sort()
lefteye.sort()
righteye.sort()
rotation_matrix.sort()
gaze.sort()
eye_coords.sort()
subject_id.sort()
data=list(zip(face,lefteye,righteye,rotation_matrix,eye_coords,subject_id,gaze))   
random.seed(12)
random.shuffle(data)
data=tf.data.experimental.from_list(data)

In [15]:
gaze_model(next(iter(data.map(map_fn_gaze).batch(1).map(lambda x,y:x))))

<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[ 0.02049856,  0.10625822, -0.9727191 ]], dtype=float32)>

### checking evaluation metrics for the new person using the updated gazemodel after calibrating the embeddings using calibration model

In [12]:
preds,labels=[],[]
for dat,label in data.map(map_fn_gaze).batch(1000):
    preds+=gaze_model(dat).numpy().tolist()
    labels+=label.numpy().tolist()

In [13]:
er=AngularError()
er.update_state(labels,preds)
er.result()

<tf.Tensor: shape=(), dtype=float32, numpy=4.869247>